# Writing a quantum program 3: addressing bits

Qualtran lets you write quantum programs and subroutines by composing lower-level subroutines, gates, and operations. We call these composable objects *bloqs* because they are the quantum building blocks of a complex algorithm. Composition is recursive: composing (lower-level) bloqs defines (higher-level) bloqs. 

In this tutorial, you will write a very simple quantum program by composing bloqs. The program will declare two input/output registers named 'x' and 'y' and swap the (quantum) integers stored within. In particular, we will swap each individual bit encoding the two integers. This tutorial will show you how `Split` and `Join` can be used to implement low-level routines.

## Writing the program

Our subroutine inputs and outputs two quantum integers. Often, we try to operate on high-level data types like this to remove classes of (quantum) type errors. When writing low-level quantum subroutines, we have to peel back the details of the encoding to write decompositions that take us closer to a typical quantum gateset of one- and two-qubit gates.

To swap two integers, we can swap each bit encoding the integers. In pseudocode, this is

```pseudocode
    input qint x, y
    for each bit in x, y:
        x[i], y[i] = bit_swap(x[i], y[i])
    output qint x, y
```

We can translate this pseudocode into a well-formed Qualtran program relatively straightforwardly. We need to take particular care when iterating over "each bit in x, y". What does this actually mean? In Qualtran, we assume that data is encoded in qubits, and writing low-level operations requires temporarily removing the quantum data type abstraction to operate on the bits directly. The programmer must explicitly state their intent to do so by including *bookkeeping* operations in the program. Below, we use `split` and `join` to manipulate individual bits.

`split` takes one `QInt` soquet (i.e. one handle to a quantum integer) and returns $n$ `QBit` soquets which can be individually manipulated.

In [ ]:
# Quantum program: QIntSwap
# This program swaps two 4-bit integers, x <--> y.
# 
# Registers:
#   x: an 8-bit quantum integer
#   y: an 8-bit quantum integer

# Start writing our program.
from qualtran import BloqBuilder
from qualtran import QInt, QBit
bb = BloqBuilder()

# Add input/output registers named 'x' and 'y'.
n = 4
x = bb.add_register('x', QInt(n))
y = bb.add_register('y', QInt(n))

# Split integers into their individual bits
xs = bb.split(x)
ys = bb.split(y)

from qualtran.bloqs.basic_gates import TwoBitSwap
for i in range(n):
    xs[i], ys[i] = bb.add(TwoBitSwap(), x=xs[i], y=ys[i])

# Join bits back to a quantum integer
x = bb.join(xs, QInt(n))
y = bb.join(ys, QInt(n))

# Finish up: map final soquets to output register names.
swap_routine = bb.finalize(x=x, y=y)

As always, we want to test that the program is indeed a valid construction.

In [ ]:
import qualtran.testing as qlt_testing
qlt_testing.assert_valid_cbloq(swap_routine)

We can view the program's compute graph. Note that the `Split` and `Join` bookkeeping operations are nodes in the compute graph. You can see that `x` is *split* from a 4-bit 'wire' into four 1-bit wires. Since our program must return `x`, `y` as integers, we make sure to join the bits back up before concluding.

In [ ]:
from qualtran.drawing import show_bloq
show_bloq(swap_routine)

Traditional quantum circuit diagrams (which we call *musical score style diagrams* for complete unambiguity) don't typically have a notion of splitting and joining wires; but in Qualtran, they will be depicted as one (e.g. 4-bit) wire ending tied to multiple (e.g. 4) one-bit wires starting.

In [ ]:
show_bloq(swap_routine, 'musical_score')

## Classical simulation

The classical simulation routines handle splits and joins as expected; and we can verify that swapping the individual bits of an integer results in the integers being swapped.

In [ ]:
swap_routine.call_classically(x=5, y=6)